In [5]:
import json
import plotly.graph_objects as go

# Load JSON data from file
with open('C:\\Users\\toazb\Documents\GitHub\scenario_visualization\data\\bounaries\old_bounds\scalar_collision\inside_edge.json', 'r') as f:
    data = json.load(f)

# Extract boundary points
boundary_points = data['boundary']['boundary_points']

# Split into x, y, z
x, y, z = zip(*boundary_points)

# Plot with Plotly
fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers+lines',
    marker=dict(size=5, color='blue'),
    line=dict(color='blue')
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Boundary Points',
        width=1000,
        height=800)

fig.show()


In [1]:
import json
import plotly.graph_objects as go

# Load JSON data from file
with open('boundary0.json', 'r') as f:
    data = json.load(f)

# Extract boundary points
boundary_points = data['boundary_points']

# Unpack x, y, z
x, y, z = zip(*boundary_points)

# Generate simple triangular mesh indices if possible
# For more complex structures, you'd need Delaunay or predefined faces
# This assumes a simple 2x2 grid for illustration (adjust as needed)
# Example for a quad split into two triangles:
i = [0, 0]
j = [1, 2]
k = [2, 3]

fig = go.Figure(data=[
    go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=i,
        j=j,
        k=k,
        color='lightblue',
        opacity=0.50
    ),
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(size=4, color='blue')
    )
])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Lattice Surface',
    width=1000,
    height=800
)

fig.show()


KeyError: 'boundary_points'

In [7]:
import json
import plotly.graph_objects as go

# Load JSON data from file
with open('boundary0.json', 'r') as f:
    data = json.load(f)

boundary_points = data['boundary_points']

# OPTIONAL: if the points are in row-major order of a grid (reshape to 2D)
# Define grid shape manually or deduce it if known
rows = 10  # e.g., 10 rows
cols = 10  # e.g., 10 columns
# assert len(boundary_points) == rows * cols, "Mismatch between points and grid shape"

# Convert flat list to 2D grid for easier indexing
grid = [boundary_points[i * cols:(i + 1) * cols] for i in range(rows)]

# Flatten x, y, z again for Mesh3d
x, y, z = zip(*[pt for row in grid for pt in row])

# Generate triangle faces
i, j, k = [], [], []
for r in range(rows - 1):
    for c in range(cols - 1):
        idx = r * cols + c
        i.extend([idx, idx + 1])
        j.extend([idx + 1, idx + cols])
        k.extend([idx + cols, idx + cols + 1])

fig = go.Figure(data=[
    go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color='lightblue',
        opacity=0.5,
        name='Lattice Surface'
    ),
    go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=2, color='blue'),
        name='Points'
    )
])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Lattice Work Surface',
    width=1000,
    height=800
)

fig.show()


In [8]:
import json
import plotly.graph_objects as go
from scipy.spatial import Delaunay
import numpy as np

# Load JSON data from file
with open('boundary0.json', 'r') as f:
    data = json.load(f)

boundary_points = np.array(data['boundary_points'])
x, y, z = boundary_points[:, 0], boundary_points[:, 1], boundary_points[:, 2]

# Triangulate the x-y plane
xy_points = np.column_stack((x, y))
tri = Delaunay(xy_points)

# tri.simplices contains indices into the original point list
fig = go.Figure(data=[
    go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=tri.simplices[:, 0],
        j=tri.simplices[:, 1],
        k=tri.simplices[:, 2],
        color='lightblue',
        opacity=0.5,
        name='Surface'
    ),
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(size=3, color='blue'),
        name='Boundary Points'
    )
])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Lattice Surface from Irregular Points',
    width=1000,
    height=800
)

fig.show()


In [15]:
import json
import numpy as np
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
from itertools import combinations

# Load JSON data
with open('boundary0.json', 'r') as f:
    data = json.load(f)

points = np.array(data['boundary_points'])
x, y, z = points[:, 0], points[:, 1], points[:, 2]

# Fit k-NN (you can tweak k)
k = 6
nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(points)
_, indices = nbrs.kneighbors(points)

# Build triangle candidates from neighbor combinations
triangle_set = set()
for i, neighbors in enumerate(indices):
    for comb in combinations(neighbors[1:], 2):  # exclude self
        tri = tuple(sorted([i, *comb]))
        triangle_set.add(tri)

# Extract triangle indices
triangles = np.array(list(triangle_set))
i, j, k = triangles[:, 0], triangles[:, 1], triangles[:, 2]

# Plot
fig = go.Figure(data=[
    go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color='lightblue',
        opacity=0.5,
        name='k-NN Surface'
    ),
    # go.Scatter3d(
    #     x=x, y=y, z=z,
    #     mode='markers',
    #     marker=dict(size=3, color='blue'),
    #     name='Points'
    # )
])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Surface Mesh via k-Nearest Neighbors',
    width=1000,
    height=800
)

fig.show()
